# 오늘수업 : SCRAPING
### crawling과 scraping의 차이점
crawling은 웹페이지 전체를 다운로드 해야함, crawling 사이즈 큼, <br>
scarping은 특정한 데이터만 가져옴, 특정부분만 추출해서 사용<br>

# 전에 만들었던 다운로드 함수

In [1]:
from urllib import parse

header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

def getDownload(url, params={}, retries=3):
    resp = None
    
    try:
        resp = requests.get(url, params=params, headers=header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and retries > 0:
            print(retries)
            resp = getDownload(url, params, retries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
            
    return resp

In [2]:
import requests
from bs4 import BeautifulSoup

# 예제사이트로 scraping 하기

In [3]:
url = "http://example.webscraping.com/places/default/index"
html = getDownload(url)
dom = BeautifulSoup(html.content, 'lxml')

### 국가명 가져오기

In [4]:
nationNameList = [_.text.strip() for _ in dom.select("#results a")]

### 이미지 가져오기

In [5]:
imgList = [requests.compat.urljoin(url, _["src"]) for _ in dom.select("#results a > img")]

for src in imgList:
    img = getDownload(src)
    with open(src.split("/")[-1], "wb") as f:
        f.write(img.content)
#     print(src.split)
#     print(html.url)
    print(img.headers["Content-Type"].split('/')[0] == "image")     # content-type을 봐야함!!
#     break

True
True
True
True
True
True
True
True
True
True


### validator.w3.org/unicorn/ - 웹페이지 문법검사 페이지 : href 같은 속성이 누락되는 것 확인 해줘야 함

# 뽐뿌 뽐뿌게시판 가져오기

In [21]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
params = {"id":"ppomppu"}
html = getDownload(url, params)

### 인코딩 확인 필수

In [7]:
html.encoding

'euc-kr'

In [17]:
html.text[:10]

'<!DOCTYPE '

### 바이트 받아서 스트링으로 바꿨다가 다시 바이트로 변환

In [20]:
html = html.content.decode("euc-kr", "ignore").encode("utf-8").decode("utf-8") 

AttributeError: 'str' object has no attribute 'content'

## 뽐뿌사이트 게시판 가져오기

In [22]:
dom = BeautifulSoup(html.content, "lxml")
# 안되면 lxml -> html.parser

In [23]:
[_.text.strip() for _ in dom.select(".list_title")]

['[티몬] 소니 시계배터리 SR626SW (360원/무료)',
 '[티몬] 패션쿨스카프(10/오늘첫구매자무료)',
 '[G9] 아이스크림 6종 40EA(12900원/0원)',
 '(끌올) [옥션] 타타와 초콜릿 케잌쿠키 2BOX (8510원/무료)',
 '[인터파크]해외 커클랜드 시그니처 퓨어 허니 2.27kg/벌꿀 2개 (42,500/무료)',
 '[위메프] 착한리필 25L 10매(18,900원/무료배송) 최종혜택가 15,120원',
 '[옥션] 토니모리 올인원 에센스 120ml 1+1 (12,750/무배)',
 '[옥션] 다이나킹 R7 로봇청소기(299,000/무배)',
 '[옥션] 립톤 아이스티 복숭아 1500g + 콜드브루 컵 (스마일클럽만 7,800원/무료배송)',
 '[티몬] 순한칫솔 초극세모 4개입 외 10원품목들 (10/무료배송데이)',
 '[SKT 초콜릿] 컬쳐 105,000원권 (95,000/무료) - 계좌이체딜',
 '[위메프] 오뚜기 육개장 매운맛(용기)86g x 24개 (9,900원 무배)',
 '영화 배심원들 1+1 롯데시네마',
 '[지마켓] 버블플랍 균일가 4900원 2개구매시 1개 증정 (4900원/무료배송)',
 '[G마켓] (사료)고연어11.3키로(84.900/무료)',
 '[옥션] 천연사이다 190ml*60캔 (11,090/무배)']

In [24]:
itemList = list()
# 행단위로 가져오기 위함 
# 정규식 뜻 : tr에서 class가 list로 시작하는 애들
for _ in dom.select("tr[class^='list']")[1:]:
    item = dict()
#     tdList = _.select("td")
    tdList = _.find_all("td", recursive=False)
    item["name"] = tdList[3].select_one("a > font").text.strip()
    item["img"] = requests.compat.urljoin(url, \
                                          tdList[3].select_one("img.thumb_border")["src"]) # 이미지 경로
    item["url"] = requests.compat.urljoin(url, \
                                      tdList[3].select_one("img.thumb_border").find_parent()["href"]) # 상세페이지 경로
    item["reg"] = tdList[4].text.strip()
    if len(tdList[5]) > 0:
        thumbs = tdList[5].text.strip().split(" - ")
        item["thumbsup"] = int(thumbs[0])
        item["thumbsdown"] = int(thumbs[1])
    else:
        item["thumbsup"] = 0
        item["thumbsdown"] = 0
    item["hit"] = tdList[6].text.strip()
    itemList.append(item)
    
    
print("아이템 개수 : ", len(itemList))
print(itemList[0])
# itemList[0]["name"]
# print(itemList[0]["name"])

아이템 개수 :  20
{'name': '[G9] 자연산 골뱅이 통조림 300g 3개 + 괄도네넴띤 2개 혼종 (16,920/0)', 'img': 'http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/4/small_316734.jpg?t=20190508213159', 'url': 'http://www.ppomppu.co.kr/zboard/view.php?id=ppomppu&page=1&divpage=55&&no=316734', 'reg': '21:36:09', 'thumbsup': 0, 'thumbsdown': 2, 'hit': '595'}


# 뽐뿌 자유게시판 가져오기

In [ ]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
params = {"id":"freeboard"}
html = getDownload(url, params)

In [ ]:
dom = BeautifulSoup(html.content, "lxml")

In [ ]:
for _ in dom.select("font.list_title")[1:]:
    print("제목 :", _.text.strip())
    detail_url = requests.compat.urljoin(url, _.find_parent()["href"])
    print("상세페이지 url :", detail_url)
    html = getDownload(detail_url)
    detail_dom = BeautifulSoup(html.content, "lxml")
    
    print("내용 :", detail_dom.select_one(".board-contents").text.strip())
    print('--------------------------------------------------------------')
    break

# 안되면 1. parser upgrade => lxml
#        2. html구조 확인

# 심화 : 댓글 가져오기

In [25]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
params = {"id":"freeboard"}
html = getDownload(url, params)
dom = BeautifulSoup(html.content, "lxml")

In [27]:
for _ in dom.select("font.list_title")[1:]:
    print("제목 :", _.text.strip())
    detail_url = requests.compat.urljoin(url, _.find_parent()["href"])
    
    print("상세페이지 url :", detail_url)
    html = getDownload(detail_url)
    detail_dom = BeautifulSoup(html.content, "lxml")
    
    if type(detail_dom) == None:
        print(detail_url, "해당 게시글이 존재하지 않습니다.")
        continue
        
    print("내용 : ", detail_dom.select_one(".board-contents").text.strip())
    
    order = 1
    if(len(detail_dom.select("[id^='commentContent_']")) > 0):
        for _ in detail_dom.select("[id^='commentContent_']"):
            print(order, "번째 댓글 : ", _.text.strip())
            order += 1
    else:
        print("댓글이 없습니다.")
    print('--------------------------------------------------------------')
#     break

# 안되면 1. parser upgrade => lxml
#        2. html구조 확인

제목 : 박원순 열받겠네요.
상세페이지 url : http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1202&no=6435728
내용 :  2년전까지만 해도 생각지도 않았던 이낙연이 딱 차기주자로 치고 나오는데
 
이걸 보고 제일 열받을 사람은 다름아닌 박원순이죠.
 
서울시장 3선하고 이번에 대선 나오려고 준비 다 해놨는데
 
지난번에 자기 밑바닥을 다 보여줘버려서 그 뒤로 같은 민주당 내에서도 차기 취급을 안하게 됐죠.
 
 
 
아마 다음 대선까지 틈만 보면 치고 나오려고 할거 같은데 지금 상태로는 쉽지 않아 보입니다.
 
무엇보다 이낙연 총리가 국회의원 - 지자체장 - 총리 까지 왠만한건 다 해본 상태인데다
 
역대 가장 능력이 출중한 총리라는 이미지가 부각되어 있으니까요.
 
특히 조곤조곤하게 자한당 두들겨패는것에 대한 카타르시스를 느끼게 한다는 측면에서 지지자들에게 확실히 어필하는듯 합니다.
 
 
 
그러고보면
 
안철수 - 안희정 - 이재명으로 연결되는 차기 유력주자들이 한순간에 나락으로 떨어지는걸 보면 참...
 
그리고 이낙연 같은 인물이 갑자기 튀어나오는걸 보면... 정말 대선이라는 건 알 수가 없네요.
 
하나 확실한건 저쪽에서는 황교안 말고 나올 인물이 없을거라는거... 그러면 뭐 저쪽은 필패일테고
 
저쪽에서도 그걸 모르지 않을테니 이것저것 수를 쓰겠죠... 
 
사실 자한당이 아닌 민주당쪽이었으면 홍정욱, 오세훈 같은 젊고 잘생긴 인물을 내세우고 포장 잘해서 어떻게든 붙여보려 할텐데
 
 저쪽에는 권력욕에 찌든 꼰대들이 많아서 아마 그렇게 되지는 않을거 같네요.
 
 그나마 나경원이 가장 유력할텐데 박근혜로 인한 능력없고 골빈 여성 정치인에 대한 혐오 + 친일 이미지 때문에 아마 안될거 같네요.
1 번째 댓글 :  자기가 깜이 안되는거죠
2 번째 댓글 :  인터넷여론만 보면 박원순이 대권감은 아닌거같은데요..
-----------------------------